In [1]:
import requests
import os
import json
import pandas as pd
import time
import pymysql

In [2]:
# Connect to the database
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='root@123',
                             db='mentegy-prod')

In [3]:
# Creating a cursor
cursor = connection.cursor()

In [4]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [5]:
%env BEARER_TOKEN= AAAAAAAAAAAAAAAAAAAAAPJZOAEAAAAA4AHhQ5g%2FgMT%2F8%2FSbrwkaATyPFCc%3D6WAGebh9bWk93cLHkDhmfVIVLiGIHqE8sQ8O2Ry7IjihDPVPis 

env: BEARER_TOKEN=AAAAAAAAAAAAAAAAAAAAAPJZOAEAAAAA4AHhQ5g%2FgMT%2F8%2FSbrwkaATyPFCc%3D6WAGebh9bWk93cLHkDhmfVIVLiGIHqE8sQ8O2Ry7IjihDPVPis


In [6]:
def auth():
    return os.environ.get("BEARER_TOKEN")


def create_url():
    return "https://api.twitter.com/2/tweets/sample/stream"

In [7]:
# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


In [8]:
def get_rules(headers, bearer_token):
    with requests.get("https://api.twitter.com/2/tweets/search/stream/rules", headers=headers) as response:
        if response.status_code != 200:
            raise Exception(
                "Cannot get rules (HTTP {}): {}".format(response.status_code, response.text)
            )
    #print(json.dumps(response.json()))
    return response.json()
    

In [9]:
def delete_all_rules(headers, bearer_token, rules):
    if rules is None or "data" not in rules:
        return None

    ids = list(map(lambda rule: rule["id"], rules["data"]))
    payload = {"delete": {"ids": ids}}
    with requests.post("https://api.twitter.com/2/tweets/search/stream/rules",headers=headers,json=payload) as response:
        if response.status_code != 200:
            raise Exception(
                "Cannot delete rules (HTTP {}): {}".format(
                    response.status_code, response.text
                )
            )
    #print(json.dumps(response.json()))
    

In [10]:
def set_rules(headers, delete, bearer_token):
    # You can adjust the rules if needed
    sample_rules = [
        {"value": "dog has:images -is:retweet" , "tag": "dog pictures"},
        # OR from:NHC_Atlantic OR from:NWSHouston OR from:NWSSanAntonio OR from:USGS_TexasRain OR from:USGS_TexasFlood
        {"value": "cat has:images -grumpy -is:retweet", "tag": "cat pictures"},
        #{"place": "country_code": "US"},
        
    ]
    payload = {"add": sample_rules}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        headers=headers,
        json=payload,
    )
    if response.status_code != 201:
        raise Exception(
            "Cannot add rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    
    #print(json.dumps(response.json()))


In [11]:
def get_stream(headers, set, bearer_token):
    
    r = ''
    while r == '':
        try:
            with requests.get("https://api.twitter.com/2/tweets/search/stream?tweet.fields=created_at&expansions=author_id&user.fields=created_at", headers=headers, stream=True,) as response:
            #print(response.status_code)
                if response.status_code != 200:
                    raise Exception(
                        "Cannot get stream (HTTP {}): {}".format(
                            response.status_code, response.text
                        )
                    )
                #iterating over response
                for response_line in response.iter_lines():
                    if response_line:
                        json_response = json.loads(response_line)
                        #converting the created_at key to datetime
                        created_at = pd.to_datetime(json_response['data']['created_at']) #converting to datetime
                        text = json_response['data']['text']
                        ps = analyzer.polarity_scores(text)
                        #forming the insert query
                        sql = "INSERT INTO `app_twitter_streams` (`created_at`, `text`, `compound_score`) VALUES (%s, %s, %s);"
                        #executing the query using the execute function
                        cursor.execute(sql,(created_at,text,ps['compound']))
                        #commit the transaction after inserting every tweet
                        connection.commit()
                        print(json_response)
        except (requests.exceptions.ConnectionError, requests.exceptions.ChunkedEncodingError) as err:
            #Continue loop after 100 ms
            time.sleep(100)
            continue   
    
        except (requests.exceptions.ConnectionError, requests.exceptions.ChunkedEncodingError) as err:

In [12]:
def main():
    bearer_token = os.environ.get("BEARER_TOKEN")
    headers = create_headers(bearer_token)
    rules = get_rules(headers, bearer_token)
    delete = delete_all_rules(headers, bearer_token, rules)
    set = set_rules(headers, delete, bearer_token)
    get_stream(headers, set, bearer_token)


In [ ]:
if __name__ == "__main__":
    main()


{'data': {'author_id': '171157663', 'created_at': '2021-06-18T15:34:49.000Z', 'id': '1405911888878309380', 'text': '@BaileyCarlin https://t.co/50o97aHjt7'}, 'includes': {'users': [{'created_at': '2010-07-26T18:05:07.000Z', 'id': '171157663', 'name': 'Becky Andress', 'username': 'BeckyAndress'}, {'created_at': '2009-08-14T02:55:51.000Z', 'id': '65544386', 'name': 'B.W. Carlin', 'username': 'BaileyCarlin'}]}, 'matching_rules': [{'id': 1405911907727548417, 'tag': 'dog pictures'}]}
{'data': {'author_id': '1193663036860633089', 'created_at': '2021-06-18T15:34:49.000Z', 'id': '1405911888484044802', 'text': '@FundyLive look at my fluffy boi please https://t.co/K3TmWZT55U'}, 'includes': {'users': [{'created_at': '2019-11-10T22:54:20.000Z', 'id': '1193663036860633089', 'name': 'kc', 'username': 'cupnudelesss'}, {'created_at': '2019-08-22T13:55:12.000Z', 'id': '1164536481567973376', 'name': 'Fundy', 'username': 'FundyLive'}]}, 'matching_rules': [{'id': 1405911907727548418, 'tag': 'cat pictures'}

{'data': {'author_id': '2547932391', 'created_at': '2021-06-18T15:35:00.000Z', 'id': '1405911934088716293', 'text': '@CWGC I regularly walk my dog through Charlesworth Cemetery in Derbyshire which is a CWGC site and just wondered about the history.  Could you confirm.  Cheers https://t.co/nP61SUvqht'}, 'includes': {'users': [{'created_at': '2014-05-14T23:34:08.000Z', 'id': '2547932391', 'name': 'david burke', 'username': 'marsupial1234'}, {'created_at': '2009-07-12T20:58:14.000Z', 'id': '56189259', 'name': 'CommonwealthWarGraves', 'username': 'CWGC'}]}, 'matching_rules': [{'id': 1405911907727548417, 'tag': 'dog pictures'}]}
{'data': {'author_id': '480425365', 'created_at': '2021-06-18T15:35:00.000Z', 'id': '1405911934654988293', 'text': 'FANCY Getting away to Spain ? Try #housesitting for 3 weeks from 5 August - 26 August in a villa with pool in #Seville #Spain. Care for 2 cats 1 dog. JOIN Housesitmatch as a #housesitter &amp; CLICK FOR INFO https://t.co/paSJSuY22c #dogsitters #catsitt

{'data': {'author_id': '3841974808', 'created_at': '2021-06-18T15:35:14.000Z', 'id': '1405911993597497345', 'text': 'Open de Catalunya 2021. Selecció pel Mundial de Bordeus. Més informació a 93 323 10 58, info@fc-culturisme.cat, https://t.co/29l5iokavZ, facebook i instagram. https://t.co/e7ceQFcW35'}, 'includes': {'users': [{'created_at': '2015-10-02T10:00:55.000Z', 'id': '3841974808', 'name': 'FCC_Culturisme', 'username': 'FCC_Culturisme'}]}, 'matching_rules': [{'id': 1405911907727548418, 'tag': 'cat pictures'}]}
{'data': {'author_id': '2363198498', 'created_at': '2021-06-18T15:35:16.000Z', 'id': '1405912000107065354', 'text': '@bartandsoul @UncleDuke1969 https://t.co/lIBzrlM56w'}, 'includes': {'users': [{'created_at': '2014-02-26T21:14:00.000Z', 'id': '2363198498', 'name': 'Marla', 'username': 'marlaZavada'}, {'created_at': '2018-02-13T01:58:47.000Z', 'id': '963230940888670208', 'name': 'Bart', 'username': 'bartandsoul'}, {'created_at': '2010-04-13T13:32:12.000Z', 'id': '132521439', 

{'data': {'author_id': '595401037', 'created_at': '2021-06-18T15:35:33.000Z', 'id': '1405912071728996358', 'text': "📢Ajuts per a la redacció de projectes per a actuacions integrals de rehabilitació en edificis existents 2021\n👷\u200d♀️S'han publicat al BOP subvencions per a la redacció de projectes i altres documents de rehabilitació en edificis residencials de l'AMB\nhttps://t.co/mOOKGUdPNu https://t.co/S6lPWKCJ1U"}, 'includes': {'users': [{'created_at': '2012-05-31T11:13:48.000Z', 'id': '595401037', 'name': 'CAATEEB', 'username': 'apabcn_cat'}]}, 'matching_rules': [{'id': 1405911907727548418, 'tag': 'cat pictures'}]}
{'data': {'author_id': '22340304', 'created_at': '2021-06-18T15:35:35.000Z', 'id': '1405912079920414729', 'text': '🐾 Fun Facts about #Dogs 🐾\n\n✔️Like and Follow for more Fun Random Facts about your #Doggie 🐕\n\n#ilovemydog #dogs #woof #dog #dogoftheday #dailybarker #dogslife #dailyhounds #lovedogs\n#doglovers\n#pets #puppies #doglover #pet #petstagram #puppylove\n#puppy

{'data': {'author_id': '1401215170706251781', 'created_at': '2021-06-18T15:35:41.000Z', 'id': '1405912105170001920', 'text': 'Needle felted dog! Custom ordered \nhttps://t.co/sjVU5N82CA\n#dogsoftwitter #dog #dogs https://t.co/BbfYVgI4fx'}, 'includes': {'users': [{'created_at': '2021-06-05T16:34:09.000Z', 'id': '1401215170706251781', 'name': 'Coastal Felting - The Home of Needle Felted Art', 'username': 'coastalfelting'}]}, 'matching_rules': [{'id': 1405911907727548417, 'tag': 'dog pictures'}]}
{'data': {'author_id': '378538986', 'created_at': '2021-06-18T15:35:40.000Z', 'id': '1405912102959607814', 'text': '@ID0N0I @stonks_dog https://t.co/1VVfQkVhz1'}, 'includes': {'users': [{'created_at': '2011-09-23T11:29:34.000Z', 'id': '378538986', 'name': 't.abcabando 🍀', 'username': 'abcabando'}, {'created_at': '2009-07-26T19:12:30.000Z', 'id': '60372896', 'name': 'CRYPTO D0N0 💎', 'username': 'ID0N0I'}, {'created_at': '2021-05-12T16:44:11.000Z', 'id': '1392520971370876929', 'name': 'DogStonks', 

...

{'data': {'author_id': '1143883362832019456', 'created_at': '2021-06-18T16:38:51.000Z', 'id': '1405928003054424065', 'text': '毎日コツコツ https://t.co/jFAr5IMgMj'}, 'includes': {'users': [{'created_at': '2019-06-26T14:07:00.000Z', 'id': '1143883362832019456', 'name': 'ウナギ犬マン', 'username': 'dog_men2'}]}, 'matching_rules': [{'id': 1405911907727548417, 'tag': 'dog pictures'}]}
{'data': {'author_id': '703583121032151040', 'created_at': '2021-06-18T16:38:52.000Z', 'id': '1405928005684236291', 'text': 'https://t.co/3dmrncojFM\nGO! 프린세스 프리큐어\nhttps://t.co/A034I45vp1'}, 'includes': {'users': [{'created_at': '2016-02-27T14:11:02.000Z', 'id': '703583121032151040', 'name': '백합짤봇', 'username': 'Yuri_Pic_Bot'}]}, 'matching_rules': [{'id': 1405911907727548417, 'tag': 'dog pictures'}]}
{'data': {'author_id': '596793109', 'created_at': '2021-06-18T16:38:57.000Z', 'id': '1405928028396478465', 'text': 'Trying to get a good picture with my cat that doesn’t like to be held. As you can see, it’s going v well ht

{'data': {'author_id': '483542178', 'created_at': '2021-06-18T16:39:21.000Z', 'id': '1405928130708082693', 'text': '@kochengfess kangen banget☹️ https://t.co/9RAsYVcGSK'}, 'includes': {'users': [{'created_at': '2012-02-05T04:10:49.000Z', 'id': '483542178', 'name': 'Indah', 'username': 'andeftaindah'}, {'created_at': '2012-07-02T14:24:56.000Z', 'id': '624785829', 'name': 'K o c h e n g F e s s 🕌🐈', 'username': 'kochengfess'}]}, 'matching_rules': [{'id': 1405911907727548418, 'tag': 'cat pictures'}]}
{'data': {'author_id': '1370428857178804225', 'created_at': '2021-06-18T16:39:22.000Z', 'id': '1405928134944382982', 'text': '$CAT 5-min chart of CAT. Awaiting Buy Signal https://t.co/FanZgemkuV'}, 'includes': {'users': [{'created_at': '2021-03-12T17:38:09.000Z', 'id': '1370428857178804225', 'name': 'Monica hesham', 'username': 'hesham_monica'}]}, 'matching_rules': [{'id': 1405911907727548418, 'tag': 'cat pictures'}]}
{'data': {'author_id': '68938438', 'created_at': '2021-06-18T16:39:25.000Z'

{'data': {'author_id': '3601825096', 'created_at': '2021-06-18T16:44:51.000Z', 'id': '1405929513507250179', 'text': 'Pierre the bear and Pat the Dog are having a Teddy Bear’s picnic &amp; so is Ms Alves class #MDYrocks https://t.co/XwG0SVFTsj'}, 'includes': {'users': [{'created_at': '2015-09-09T20:10:44.000Z', 'id': '3601825096', 'name': 'St. Marguerite', 'username': 'SMCDSB_MDY'}]}, 'matching_rules': [{'id': 1405911907727548417, 'tag': 'dog pictures'}]}
{'data': {'author_id': '426437791', 'created_at': '2021-06-18T16:44:51.000Z', 'id': '1405929512240439298', 'text': 'Adorable Kids and Dogs Friends Japan\nhttps://t.co/FQukaNH5UG\n#funny https://t.co/Wb84aCXqx2'}, 'includes': {'users': [{'created_at': '2011-12-02T07:17:44.000Z', 'id': '426437791', 'name': 'Funnyness', 'username': 'FunnyNeel'}]}, 'matching_rules': [{'id': 1405911907727548417, 'tag': 'dog pictures'}]}
{'data': {'author_id': '455014974', 'created_at': '2021-06-18T16:45:00.000Z', 'id': '1405929549880250369', 'text': '🗓 Agen

{'data': {'author_id': '839817126563229697', 'created_at': '2021-06-18T16:45:13.000Z', 'id': '1405929606532763654', 'text': "✍️ Aprovat el text definitiu del #Reglament de serveis d'atenció domiciliària: serveis d'ajuda a domicili i #Teleassistència de l' @AjArenys\n🔹https://t.co/raCOKqZYf7\n\n@ccmaresme #NormativaLocal #AtencióDomiciliària https://t.co/VIx9m28SC5"}, 'includes': {'users': [{'created_at': '2017-03-09T12:36:20.000Z', 'id': '839817126563229697', 'name': 'CIDO', 'username': 'cidodiba'}, {'created_at': '2011-07-26T12:09:01.000Z', 'id': '342678361', 'name': 'Aj. Arenys de Mar', 'username': 'AjArenys'}, {'created_at': '2013-03-28T11:17:43.000Z', 'id': '1310260495', 'name': 'CC Maresme', 'username': 'ccmaresme'}]}, 'matching_rules': [{'id': 1405911907727548418, 'tag': 'cat pictures'}]}
{'data': {'author_id': '27887540', 'created_at': '2021-06-18T16:45:18.000Z', 'id': '1405929624979312648', 'text': "You were a good cat Maxwell. I'm going to miss you bud. https://t.co/4tLd17AaE6

{'data': {'author_id': '1166647050', 'created_at': '2021-06-18T16:45:36.000Z', 'id': '1405929699822510093', 'text': '@seguin_nathalie Sûrement la faute de la vente hot-dog du ministre Dubé et ces faux vaccin https://t.co/jAMoeb7htr'}, 'includes': {'users': [{'created_at': '2013-02-10T18:35:23.000Z', 'id': '1166647050', 'name': 'johnny  batailleur', 'username': 'jeanbattailleur'}, {'created_at': '2012-11-03T04:53:39.000Z', 'id': '922466882', 'name': 'Séguin Nathalie ⚜️', 'username': 'seguin_nathalie'}]}, 'matching_rules': [{'id': 1405911907727548417, 'tag': 'dog pictures'}]}
